# **Análisis Exploratorio de Datos (EDA)**

## **Librerías y modulos necesarios**

In [7]:
import pandas as pd

## **Importación de datos**

Como se señaló en la sección de [contextualización](./contextualizacion.md), los datos utilizados provienen de la competencia [Give Me Some Credit](https://www.kaggle.com/competitions/GiveMeSomeCredit), desde donde pueden descargarse. Para facilitar el trabajo reproducible en este proyecto, hemos alojado los conjuntos de entrenamiento y prueba proporcionados por la competencia en el repositorio de GitHub [sePerezAlbor/Data](https://github.com/sePerezAlbor/Data), específicamente los archivos `GiveMeSomeCredit-cs-training.csv` y `GiveMeSomeCredit-cs-test.csv`. Para importar los datos se recomienza utilizar el siguiente código.

In [8]:
##### TRAINING DATA
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/GiveMeSomeCredit-cs-training.csv"
train_data = pd.read_csv(url, na_values="NA", index_col=0)

##### TEST DATA
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/GiveMeSomeCredit-cs-test.csv"
test_data = pd.read_csv(url, na_values="NA", index_col=0)

## **Renombrar columnas**

Esto es opcional y puede omitirse si se prefiere trabajar con los nombres de columnas originales.

In [9]:
train_data = train_data.rename(columns={
    'SeriousDlqin2yrs': 'mora_grave_2anios',
    'RevolvingUtilizationOfUnsecuredLines': 'utilizacion_lineas_credito',
    'age': 'edad',
    'NumberOfTime30-59DaysPastDueNotWorse': 'veces_mora_30_59_dias_2anios',
    'DebtRatio': 'razon_deuda_ingreso',
    'MonthlyIncome': 'ingreso_mensual',
    'NumberOfOpenCreditLinesAndLoans': 'num_creditos_lineas_abiertas',
    'NumberOfTimes90DaysLate': 'veces_mora_90_dias',
    'NumberRealEstateLoansOrLines': 'num_prestamos_inmobiliarios',
    'NumberOfTime60-89DaysPastDueNotWorse': 'veces_mora_60_89_dias_2anios',
    'NumberOfDependents': 'num_dependientes'
})

## **Diccionario de variables**

```{list-table} Diccionario de variables Give Me Some Credit
:header-rows: 1
:name: tabla-diccionario-gmsc

* - Variable
  - Variable Renombrada
  - Descripción
  - Tipo
* - `SeriousDlqin2yrs`
  - `mora_grave_2anios`
  - Persona con morosidad de 90 días o más.
  - Y/N
* - `RevolvingUtilizationOfUnsecuredLines`
  - `utilizacion_lineas_credito`
  - Saldo total de tarjetas de crédito y líneas de crédito personales, excepto bienes inmuebles y deudas sin cuotas fijas, como préstamos para automóviles, dividido por la suma de los límites de crédito.
  - percentage
* - `age`
  - `edad`
  - Edad del prestatario en años.
  - integer
* - `NumberOfTime30-59DaysPastDueNotWorse`
  - `veces_mora_30_59_dias_2anios`
  - Número de veces que el prestatario ha tenido un retraso de entre 30 y 59 días, pero sin agravarse, en los últimos dos años.
  - integer
* - `DebtRatio`
  - `razon_deuda_ingreso`
  - Pagos mensuales de deudas, pensión alimenticia y gastos de manutención divididos entre los ingresos mensuales.
  - percentage
* - `MonthlyIncome`
  - `ingreso_mensual`
  - Ingresos mensuales.
  - real
* - `NumberOfOpenCreditLinesAndLoans`
  - `num_creditos_lineas_abiertas`
  - Número de préstamos abiertos (préstamos a plazos, como préstamos para automóviles o hipotecas) y líneas de crédito (por ejemplo, tarjetas de crédito).
  - integer
* - `NumberOfTimes90DaysLate`
  - `veces_mora_90_dias`
  - Número de veces que el prestatario ha acumulado un retraso de 90 días o más en el pago.
  - integer
* - `NumberRealEstateLoansOrLines`
  - `num_prestamos_inmobiliarios`
  - Número de préstamos hipotecarios e inmobiliarios, incluidas las líneas de crédito con garantía hipotecaria.
  - integer
* - `NumberOfTime60-89DaysPastDueNotWorse`
  - `veces_mora_60_89_dias_2anios`
  - Número de veces que el prestatario ha tenido un retraso de entre 60 y 89 días, pero sin agravarse, en los últimos dos años.
  - integer
* - `NumberOfDependents`
  - `num_dependientes`
  - Número de personas a cargo en la familia, excluyéndose a sí mismos (cónyuge, hijos, etc.).
  - integer


# **Conjunto de datos**

In [10]:
train_data.head() 

,mora_grave_2anios,utilizacion_lineas_credito,edad,veces_mora_30_59_dias_2anios,razon_deuda_ingreso,ingreso_mensual,num_creditos_lineas_abiertas,veces_mora_90_dias,num_prestamos_inmobiliarios,veces_mora_60_89_dias_2anios,num_dependientes
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


Se observa que las variables abarcan tanto características financieras como demográficas, incluyendo la edad del solicitante y el número de dependientes, así como múltiples indicadores relacionados con su comportamiento crediticio, tales como la utilización de líneas de crédito, la frecuencia de moras en distintos rangos de días, la relación deuda–ingreso, el ingreso mensual y el número de préstamos o líneas de crédito abiertas. Esta combinación de atributos, descritos en detalle en el [diccionario de variables Give Me Some Credit](tabla-diccionario-gmsc), proporciona una base integral para analizar patrones de riesgo y establecer relaciones entre factores personales y financieros que pueden influir en la probabilidad de morosidad grave.
